In [19]:
# Import data handling libraries
import pandas as pd
import numpy as np

#plot the distribution of GRAD_SEM
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
#read in the raw dataset
df = pd.read_csv('data_set_10_17.csv')

#a simple function to get info about the dataset
def gradestats(df):
    total = df["STUDENT"].nunique()
    grads = df.groupby("STUDENT")["DEG_CD"].apply(lambda x: x.notnull().any()).sum()
    percent = np.round(grads/total*100,2)
    print('Rows:', len(df))
    print('Total:', total)
    print('Grads:', grads)
    print('Percent:', percent)
    print()

gradestats(df)

Rows: 30394
Total: 13065
Grads: 7622
Percent: 58.34



In [21]:
#remove students who entered before 2008 (because the data is shaky)
df = df[df['ENTRY_CCYY'] >= 2008]

print('After removing students who entered before 2008:')
gradestats(df)

After removing students who entered before 2008:
Rows: 30219
Total: 13013
Grads: 7578
Percent: 58.23



In [22]:
#dropped degrees are graduate degrees. kept degrees are undergraduate degrees.
dropped_degrees = ['MRE', 'MCP', 'MED', 'MFA', 'MTR', 'MHC', 'MFS', 'MHO', 'MAA', 'DVM', 'MBS', 'MBA', 'MA', 'MEN', 'MFN', 'MAT', 'PHD', 'MS']

#here I drop the graduate degrees
for x in dropped_degrees:
    df = df.drop(df[df['DEG_CD'] == x].index)
df.reset_index(drop=True, inplace=True)
print('After dropping graduate degrees')
gradestats(df)

After dropping graduate degrees
Rows: 28992
Total: 13000
Grads: 7565
Percent: 58.19



In [23]:
#Next, we format the semester in which the student entered the university, took the course, and obtained a degree

#subtract 2008 from 'ENTRY_CCYY', 'SEM_CCYY', and 'SEM_CCYY.1'
for x in ['ENTRY_CCYY', 'SEM_CCYY', 'SEM_CCYY.1']:
    df[x] = df[x] - 2000

#encode df['ENTRY_SEM_CD] as {'F':0.6,'S':0.2,1:0.4}
df['ENTRY_SEM_CD'] = df['ENTRY_SEM_CD'].replace({'F':0.6,
                                                 'S':0.0,
                                                 '1':0.4})

#Next, we encode the semester in which the course was taken and the semester in which the student graduated
for sem in ['SEM_CD', 'SEM_CD.1']:
    df[sem] = df[sem].replace({'F': 1.0, 
                               '1': 0.6, 
                               'S': 0.4})

#create columns for entry, course, and grad semesters.
df['ENT_SEM'] = df['ENTRY_CCYY'] + df['ENTRY_SEM_CD']
df['CRSE_SEM'] = df['SEM_CCYY'] + df['SEM_CD']
df['GRAD_SEM'] = df['SEM_CCYY.1'] + df['SEM_CD.1']

#Remove students who enrolled at least in Summer 2020 and did not graduate.
df = df.drop(df[(df['ENT_SEM'] >= 20.4) & (df['GRAD_SEM'].isnull())].index)

print('After removing students who enrolled later than Spring 2020 and did not yet graduate:')
gradestats(df)

#drop all rows for which CRSE_SEM is greater than 8 or greater than GRAD_SEM (if this occurs, it means that the student took the course after they graduated; this scenario likely happened because the student continued taking courses for a second degree)
df = df[(df['GRAD_SEM'].isnull()) | (df['CRSE_SEM'] <= df['GRAD_SEM'])]

print('After removing courses taken after graduation:')
gradestats(df)

After removing students who enrolled later than Spring 2020 and did not yet graduate:


/var/folders/8h/w7q1fjwd5kn8xvqty5sp3h7r0000gn/T/ipykernel_88981/3271962143.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ENTRY_SEM_CD'] = df['ENTRY_SEM_CD'].replace({'F':0.6,
/var/folders/8h/w7q1fjwd5kn8xvqty5sp3h7r0000gn/T/ipykernel_88981/3271962143.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[sem] = df[sem].replace({'F': 1.0,


Rows: 23340
Total: 9795
Grads: 7565
Percent: 77.23

After removing courses taken after graduation:
Rows: 23171
Total: 9766
Grads: 7536
Percent: 77.17



In [24]:
check = df[['STUDENT',
                'CRSE',
                'SEM_CCYY',
                'SEM_CD'
                ]]
#check how many duplicate rows are there in the dataset
check.duplicated().sum()

np.int64(1436)

In [25]:
"""Output: np.int64(1436)"""

#get the indices of the duplicate rows and drop them from the 'df' dataset
df.drop(index= check[check.duplicated()].index, inplace=True)

print('After dropping duplicates:')
gradestats(df)

After dropping duplicates:
Rows: 21735
Total: 9766
Grads: 7536
Percent: 77.17



In [26]:
#remove the 'X' and 'H' appearing in any CRSE name
df['CRSE'] = df['CRSE'].str.replace('X', '')
df['CRSE'] = df['CRSE'].str.replace('H', '')

#remove all courses for which CRSE_TITLE contains 'EDUC'
df = df[~df['CRSE_TITLE'].str.contains('EDUC')]

#the courses x = '302' and '403' both refer to abstract algebra II, so let's combine them and call them '302'
#these are supposed to be the same, so re-number the '403' to '302'
df['CRSE'] = df['CRSE'].replace('403', '302')

#the courses x = '142' and '145' are both variations of trig, so let's combine them and call them '142'
df['CRSE'] = df['CRSE'].replace('145', '142')

#the course x='407' is applied linear algebra; very similar to '317', so let's combine them and call them '317'
df['CRSE'] = df['CRSE'].replace('407', '317')

#the courses x='497' and '397' are both about teaching secondary school math, we combine them and call them '397'
df['CRSE'] = df['CRSE'].replace('497', '397')

#it's natural to combine x = '490' (Independent Study) with x='495' (Special topics) and call them '495'
df['CRSE'] = df['CRSE'].replace('490', '495')

#cast the course numbers as integers
df['CRSE'] = df['CRSE'].astype(int)

#replace all courses >= 500 with 500 (grad courses)
df['CRSE'] = np.where(df['CRSE'] >= 500, 500, df['CRSE'])

#for any row with CRSE = 500, replace the CRSE_TITLE with 'GRADUATE MATH'
df['CRSE_TITLE'] = np.where(df['CRSE'] == 500, 'GRADUATE MATH', df['CRSE_TITLE'])

#if any CRSE corresponds to more than one CRSE_TITLE, then replace all CRSE_TITLES with the first one that appears
df['CRSE_TITLE'] = df.groupby('CRSE')['CRSE_TITLE'].transform('first')

#if any CRSE_TITLE corresponds to more than one CRSE, then replace all CRSEs with the first one that appears
df['CRSE'] = df.groupby('CRSE_TITLE')['CRSE'].transform('first')

#remove courses with less than 50 students
df = df.groupby('CRSE').filter(lambda x: len(x) >= 50)

print('After formatting courses, and removing courses with low enrollment:')
gradestats(df)

After formatting courses, and removing courses with low enrollment:
Rows: 21361
Total: 9710
Grads: 7490
Percent: 77.14



In [27]:
crse_counts = df.groupby(by=['CRSE','CRSE_TITLE']).size().reset_index(name='COUNT').sort_values(by='COUNT', ascending=False).reset_index(drop=True)
crse_counts.to_csv('crse_counts.csv', index=False)
crse_counts

,CRSE,CRSE_TITLE,COUNT
0,165,CALCULUS I,2756
1,166,CALCULUS II,2325
2,265,CALCULUS III,1682
3,143,PREP FOR CALCULUS,1507
4,140,COLLEGE ALGEBRA,1373
5,104,INTRO TO PROBABILTY,1252
6,150,DISC MATH BUS&SOC S,1110
7,207,MATRCES&LINEAR ALGB,949
8,201,INTRO TO PROOFS,758
9,317,THRY LINEAR ALGEBRA,738


In [28]:
crse_counts.sort_values(by='CRSE',inplace=True)

#make a dict of CRSE : CRSE_TITLE pairs
crse_counts['CRSE'] = crse_counts['CRSE'].astype(str)
courses = crse_counts['CRSE'].unique().tolist()
crse_dict = dict(zip(crse_counts['CRSE'], crse_counts['CRSE_TITLE']))
crse_dict

{'25': 'HIGH SCHOOL ALGEBRA',
 '101': 'ORIENTATION IN MATH',
 '104': 'INTRO TO PROBABILTY',
 '105': 'INTRO TO MATH IDEAS',
 '106': 'DISCOVERING MATH',
 '140': 'COLLEGE ALGEBRA',
 '142': 'TRIG&ANALY GEOMETRY',
 '143': 'PREP FOR CALCULUS',
 '150': 'DISC MATH BUS&SOC S',
 '151': 'BUS & SOC SCI CALC',
 '160': 'SURVEY OF CALCULUS',
 '165': 'CALCULUS I',
 '166': 'CALCULUS II',
 '181': 'LIFE SCI CALC&MDL I',
 '201': 'INTRO TO PROOFS',
 '207': 'MATRCES&LINEAR ALGB',
 '240': 'INVESTM&CREDIT MATH',
 '265': 'CALCULUS III',
 '266': 'ELEM DIFF EQUATNS',
 '267': 'DIFF EQ & TRANSFMS',
 '301': 'ABSTRACT ALGEBRA I',
 '302': 'ABSTRCT ALGEBRA II',
 '304': 'COMBINATORICS',
 '314': 'GRAPH THEORY',
 '317': 'THRY LINEAR ALGEBRA',
 '341': 'INTR THY PROBAB&S I',
 '342': 'INTR THY PR&STAT II',
 '350': 'NUMBER THEORY',
 '365': 'COMPLEX VARIABLES',
 '373': 'INTR SCIENTF CMPTNG',
 '385': 'INTR PART DIFF EQUA',
 '397': 'UNIV MATH TCH SECDY',
 '414': 'ANALYSIS I',
 '415': 'ANALYSIS II',
 '435': 'GEOMETRY I',
 '436':

In [29]:
#format the grades
df['GRADE_CATGORY'] = df['GRADE_CATGORY'].map({'C- OR BELOW':-1,
                                               'ABOVE C-':1,
                                               'Satisfactory/Pass':1})

df['CRSE'] = df['CRSE'].astype(str)
#We add one column for each course and temporarily stick a 0 in each column
for x in courses:
    df[x] = 0.0

#subtract ENT_SEM from CRSE_SEM and GRAD_SEM to get relative semester values
df['CRSE_SEM'] = df['CRSE_SEM'] - df['ENT_SEM']
df['GRAD_SEM'] = df['GRAD_SEM'] - df['ENT_SEM']

# #rpelace grad_time with 3.8/grad_time. This is because the longer it takes to graduate, the less likely the student is to graduate. This also introduces an element of continuity in the values when we fill in the nan values (non-graduates) with 0.
# df['GRAD_SEM'] = 3.8/df['GRAD_SEM']
# df['GRAD_SEM'].fillna(0, inplace=True)

#Now, in each row, if a student has taken a course x, then the value in the column 'x' is replaced with the product of the 'GRADE_CATEGORY' and 'CRSE_SEM' 
for x in courses:
    df.loc[df['CRSE'] == x, x] = df['GRADE_CATGORY'] * df['CRSE_SEM']

In [30]:
cols = df.columns.tolist()
cols

"""Output:
['STUDENT',
 'MAJOR_CURR',
 'MAJOR_COLL',
 'CLSFN_YR',
 'ENTRY_SEM_CD',
 'ENTRY_CCYY',
 'LEVEL',
 'SEM_CCYY',
 'SEM_CD',
 'OFFER_DEPT_ABRVN',
 'CRSE',
 'SECT',
 'GRADE_CATGORY',
 'CRSE_TITLE',
 'SEM_CCYY.1',
 'SEM_CD.1',
 'DEG_CD',
 'DEG_DATEDB',
 'MAJOR_CURR.1',
 'DEG_DESCR',
 'LAST_RGST_TERM',
 'CLSFN_YR.1',
 'MAJOR_CURR.2',
 'MAJOR_COLL.1',
 'Workday Enrolled in Fall 2024',
 'Workday Enrolled in Fall 2024 Class Standing',
 'Workday Enrolled in Fall 2024 Primary Program of Study',
 'ENT_SEM',
 'CRSE_SEM',
 'GRAD_SEM',
 '25',
 '101',
 '104',
 '105',
 '106',
 '140',
 '142',
 '143',
 '150',
 '151',
 '160',
 '165',
 '166',
 '181',
 '201',
 '207',
 '240',
 '265',
 '266',
 '267',
 '301',
 '302',
 '304',
 '314',
 '317',
 '341',
 '342',
 '350',
 '365',
 '373',
 '385',
 '397',
 '414',
 '415',
 '435',
 '436',
 '492',
 '495',
 '500']"""

"Output:\n['STUDENT',\n 'MAJOR_CURR',\n 'MAJOR_COLL',\n 'CLSFN_YR',\n 'ENTRY_SEM_CD',\n 'ENTRY_CCYY',\n 'LEVEL',\n 'SEM_CCYY',\n 'SEM_CD',\n 'OFFER_DEPT_ABRVN',\n 'CRSE',\n 'SECT',\n 'GRADE_CATGORY',\n 'CRSE_TITLE',\n 'SEM_CCYY.1',\n 'SEM_CD.1',\n 'DEG_CD',\n 'DEG_DATEDB',\n 'MAJOR_CURR.1',\n 'DEG_DESCR',\n 'LAST_RGST_TERM',\n 'CLSFN_YR.1',\n 'MAJOR_CURR.2',\n 'MAJOR_COLL.1',\n 'Workday Enrolled in Fall 2024',\n 'Workday Enrolled in Fall 2024 Class Standing',\n 'Workday Enrolled in Fall 2024 Primary Program of Study',\n 'ENT_SEM',\n 'CRSE_SEM',\n 'GRAD_SEM',\n '25',\n '101',\n '104',\n '105',\n '106',\n '140',\n '142',\n '143',\n '150',\n '151',\n '160',\n '165',\n '166',\n '181',\n '201',\n '207',\n '240',\n '265',\n '266',\n '267',\n '301',\n '302',\n '304',\n '314',\n '317',\n '341',\n '342',\n '350',\n '365',\n '373',\n '385',\n '397',\n '414',\n '415',\n '435',\n '436',\n '492',\n '495',\n '500']"

In [31]:
final_cols = \
['STUDENT',
 'ENT_SEM',
 'GRAD_SEM',
 '25',
 '101',
 '104',
 '105',
 '106',
 '140',
 '142',
 '143',
 '150',
 '151',
 '160',
 '165',
 '166',
 '181',
 '201',
 '207',
 '240',
 '265',
 '266',
 '267',
 '301',
 '302',
 '304',
 '314',
 '317',
 '341',
 '342',
 '350',
 '365',
 '373',
 '385',
 '397',
 '414',
 '415',
 '435',
 '436',
 '492',
 '495',
 '500']
df = df[final_cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21361 entries, 0 to 28991
Data columns (total 42 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   STUDENT   21361 non-null  int64  
 1   ENT_SEM   21361 non-null  float64
 2   GRAD_SEM  17786 non-null  float64
 3   25        21361 non-null  float64
 4   101       21361 non-null  float64
 5   104       21361 non-null  float64
 6   105       21361 non-null  float64
 7   106       21361 non-null  float64
 8   140       21361 non-null  float64
 9   142       21361 non-null  float64
 10  143       21361 non-null  float64
 11  150       21361 non-null  float64
 12  151       21361 non-null  float64
 13  160       21361 non-null  float64
 14  165       21361 non-null  float64
 15  166       21361 non-null  float64
 16  181       21361 non-null  float64
 17  201       21361 non-null  float64
 18  207       21361 non-null  float64
 19  240       21361 non-null  float64
 20  265       21361 non-null  float64

In [32]:
#replace all null values with 0
df = df.fillna(0)

#group by student, choose the max value in each column
df = df.groupby('STUDENT').agg(lambda x: x.loc[x.abs().idxmax()])

#Add a column 'y' whose value is 0 if 'GRAD_SEM' is 0 or greater than 8, and 1 otherwise
df['Y'] = 0
df['Y'][(df['GRAD_SEM'] > 0) & (df['GRAD_SEM'] <= 3.8)] = 1

print('After aggregating the dataset by student and adding the target variable:')
df

After aggregating the dataset by student and adding the target variable:


/var/folders/8h/w7q1fjwd5kn8xvqty5sp3h7r0000gn/T/ipykernel_88981/93202105.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Y'][(df['GRAD_SEM'] > 0) & (df['GRAD_SEM'] <= 3.8)] = 1
/var/folders/8h/w7q1fjwd5kn8xvqty5sp3h7r0000gn/T/ipyker

,ENT_SEM,GRAD_SEM,25,101,104,105,106,140,142,143,...,385,397,414,415,435,436,492,495,500,Y
STUDENT,,,,,,,,,,,,,,,,,,,,,
0,20.6,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,19.6,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,19.6,3.4,0.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9,16.6,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
10,18.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13060,14.6,4.8,0.0,0.0,0.0,0.0,0.0,-0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
13061,16.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
13062,13.6,2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [33]:
df['165'].value_counts()

165
 0.0    7165
 0.4     768
 0.8     371
 0.4     323
 0.8     137
        ... 
-0.2       1
 1.2       1
-1.2       1
-2.8       1
 3.6       1
Name: count, Length: 97, dtype: int64

In [34]:
#scale all course columns by 2.5 (so fall and spring count as 1 sem, and summer counts as 0.5)
df[courses] = df[courses].apply(lambda x: np.round(x*2.5,1))

In [35]:
df['165'].value_counts()

165
 0.0     7165
 1.0     1091
 2.0      585
-1.0      137
-2.0      114
 4.5       89
 3.5       83
 2.5       76
 5.0       36
-3.5       35
 6.0       30
 7.0       26
 0.5       25
-4.5       23
 1.5       22
 9.5       21
 7.5       19
-2.5       12
-7.0       11
 8.5       11
 10.0      10
 4.0        8
-5.0        7
-6.0        7
-8.5        6
 12.0       5
 12.5       5
-1.5        5
-0.5        3
 11.0       3
-7.5        3
-11.0       3
-3.0        3
-9.5        3
 13.5       3
 16.0       2
 3.0        2
 5.5        2
 15.0       2
 17.0       2
 6.5        2
-10.0       2
 23.5       1
-8.0        1
 25.5       1
 21.5       1
-4.0        1
-13.5       1
 22.5       1
 11.5       1
 16.5       1
-5.5        1
 9.0        1
Name: count, dtype: int64

In [36]:
#get a list of all possible values in df[courses]
crse_values = [df[x].unique().tolist() for x in courses]

#flatten the list
crse_values = [item for sublist in crse_values for item in sublist]
crse_values = list(set(crse_values))
crse_values.sort()

# Create a DataFrame for SEM columns
sem_df = pd.concat([df[courses].apply(lambda x: x.tolist().count(i) - x.tolist().count(-i), axis=1).rename(f'SEM{i}') for i in crse_values], axis=1)

# Add the SEM columns to the original DataFrame
df = pd.concat([df, sem_df], axis=1)

# Define columns for the cumulative sums of math performance
df[[f'SEM{i}_cdf' for i in crse_values]] = sem_df.cumsum(axis=1)

df['slope'] = 0
df['intercept'] = 0

# Loop through the rows of the DataFrame
x = crse_values
for i, row in df.iterrows():
    #y values
    y = row[[f'SEM{j}_cdf' for j in crse_values]].astype(float)
    # Calculate the slope and intercept
    S,I = np.polyfit(x, y, 1)
    # Store the values in the DataFrame
    df.at[i, 'slope'] = S
    df.at[i, 'intercept'] = I

/var/folders/8h/w7q1fjwd5kn8xvqty5sp3h7r0000gn/T/ipykernel_88981/1122137119.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.0026321085934822835' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'slope'] = S
/var/folders/8h/w7q1fjwd5kn8xvqty5sp3h7r0000gn/T/ipykernel_88981/1122137119.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.15702410362977462' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'intercept'] = I


In [37]:
#save to csv
df.to_csv('final_dataset_Nov_24.csv', index=False)

In [40]:
np.max(crse_values), np.min(crse_values)

(np.float64(32.0), np.float64(-24.5))